In [1]:
import pandas as pd
import sqlite3

In [14]:
dir_db = 'sql-murder-mystery.db'
conn = sqlite3.connect(dir_db)
cursor =  conn.cursor()

In [20]:
query = '''
SELECT description
FROM crime_scene_report
WHERE date = '20180115' and type = 'murder' and city = 'SQL City'
'''

pd.read_sql_query(query, conn).values

array([['Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".']],
      dtype=object)

In [26]:
query = '''
SELECT *
FROM person
WHERE address_street_name = "Northwestern Dr"
ORDER BY address_number DESC
'''

pd.read_sql_query(query, conn).head(1)

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [33]:
query = '''
SELECT *
FROM person
WHERE address_street_name = "Franklin Ave"
'''

pd.read_sql_query(query, conn).iloc[1]

id                              16371
name                   Annabel Miller
license_id                     490173
address_number                    103
address_street_name      Franklin Ave
ssn                         318771143
Name: 1, dtype: object

In [45]:
query = '''
SELECT p.name, int.*
FROM person as p
JOIN interview int ON p.Id = int.person_Id
WHERE p.name = "Annabel Miller" or p.name = 'Morty Schapiro'

'''

pd.read_sql_query(query, conn).transcript.values

array(['I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".',
       'I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.'],
      dtype=object)

In [66]:
clues = '''
gym = "Get Fit Now Gym" / gold member
gym membership = "48Z%"
car plate = "%H42W%"
day = "January 9th"
'''

query = '''
SELECT *
FROM get_fit_now_member
WHERE membership_status = 'gold' and id LIKE '48Z%'
'''

pd.read_sql_query(query, conn)

,id,person_id,name,membership_start_date,membership_status
0,48Z7A,28819,Joe Germuska,20160305,gold
1,48Z55,67318,Jeremy Bowers,20160101,gold


In [70]:
query = '''
SELECT g.*, ci.check_in_date
FROM get_fit_now_member g
JOIN get_fit_now_check_in ci ON g.Id = ci.membership_Id
WHERE g.membership_status = 'gold' and g.id LIKE '48Z%' 
'''

pd.read_sql_query(query, conn)

,id,person_id,name,membership_start_date,membership_status,check_in_date
0,48Z7A,28819,Joe Germuska,20160305,gold,20180109
1,48Z55,67318,Jeremy Bowers,20160101,gold,20180109


In [76]:
query = '''
SELECT g.id, g.name, g.membership_status, ci.check_in_date, dl.plate_number
FROM get_fit_now_member g
JOIN get_fit_now_check_in ci ON g.Id = ci.membership_Id
JOIN person p ON g.person_Id = p.Id
JOIN drivers_license dl ON p.license_Id = dl.Id
WHERE g.membership_status = 'gold' and g.id LIKE '48Z%' and dl.plate_number LIKE "%H42W%"
'''

pd.read_sql_query(query, conn)

,id,name,membership_status,check_in_date,plate_number
0,48Z55,Jeremy Bowers,gold,20180109,0H42W2
